## Exercise 1
Some simple plotting

## Exercise 2
Some more advanced plotting

## Exercise 3
- Computing OLS by matrix inversion
- Plotting OLS and data
- Comparing to scipy regression function

## Exercise 4
Some optimization problem